In [ ]:
import cv2
import numpy as np
import tensorflow as tf
from pygame import mixer
from keras.models import load_model

In [ ]:
MODEL_V1 = r'models/model_v1.h5'
MODEL_V2 = r'models/model_v2.h5'

In [ ]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
model = load_model(MODEL_V2)


In [ ]:
mixer.init()
sound = mixer.Sound(r'models/assets/alarm.wav')
cap = cv2.VideoCapture(0)
Score = 0

In [ ]:
while True:
    ret, frame = cap.read()
    height, width = frame.shape[0:2]

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=3)
    eyes = eye_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=1)

    cv2.rectangle(frame, (0, height-50), (200, height), (0, 0, 0), thickness=cv2.FILLED)

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, pt1=(x, y), pt2=(x+w, y+h), color=(255, 0, 0), thickness=3)

        for (ex, ey, ew, eh) in eyes:
            eye = frame[ey:ey+eh, ex:ex+w]
            eye = cv2.resize(eye, (80, 80))
            eye = eye / 255
            eye = eye.reshape(80, 80, 3)
            eye = np.expand_dims(eye, axis=0)
            prediction = model.predict(eye)

            #Condition for Drowsy
            if prediction[0][0] > 0.30:
                cv2.putText(frame, 'Drowsy', (10, height-20), fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL, fontScale=1,
                            color=(255, 255, 255), thickness=1, lineType=cv2.LINE_AA)
                cv2.putText(frame, 'Score'+str(Score), (150, height-20), fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,
                            fontScale=1, color=(255, 255, 255), thickness=1, lineType=cv2.LINE_AA)
                Score = Score + 1

                if Score < 10:
                    try:
                        sound.play()
                    except:
                        pass

            #Condition for Awake                
            elif prediction[0][1] > 0.20:
                cv2.putText(frame, 'Awake', (10, height-20), fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL, fontScale=1,
                            color=(255, 255, 255), thickness=1, lineType=cv2.LINE_AA)
                cv2.putText(frame, 'Score'+str(Score), (150, height-20), fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,
                            fontScale=1, color=(255, 255, 255), thickness=1, lineType=cv2.LINE_AA)
                Score = Score - 1

                if Score < 0:
                    Score = 0
        
    cv2.imshow('StayAwake', frame)
    if cv2.waitKey(33) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
